In [ ]:
import os
import json

In [ ]:
def json_to_yolo(json_file, output_folder, label_mapping, image_folder):
    
    with open(json_file) as f:
        data = json.load(f)

    
    image_info = data["images"]
    image_name = image_info["file_name"].split('.')[0]
    yolo_labels = []

    
    for annotation_info in data["annotations"]:
       
        category_id = annotation_info["category_id"]

        if category_id in label_mapping:
            bbox = annotation_info["bbox"]  
            xmin, ymin, width, height = bbox  

            
            class_id = label_mapping[category_id]

            try:
                image_info["width"] = 1280
                image_info["height"] = 720
                # 카테고리와 관련된 좌표를 YOLO format으로 변환합니다.
                x_center = (xmin + width / 2) / image_info["width"]
                y_center = (ymin + height / 2) / image_info["height"]
                box_width = width / image_info["width"]
                box_height = height / image_info["height"]
            except ZeroDivisionError:
                print("오류")
                return

            yolo_labels.append(f"{class_id} {x_center} {y_center} {box_width} {box_height}")

    output_file_path = os.path.join(output_folder, f"{image_name}.txt")

    with open(output_file_path, "w") as f:
        f.write("\n".join(yolo_labels))

    if os.path.getsize(output_file_path) == 0:
        os.remove(output_file_path)
        
        image_file_path = None
        for image_ext in [".png"]:
            test_path = os.path.join(image_folder, f"{image_name}{image_ext}")
            if os.path.exists(test_path):
                image_file_path = test_path
                break
        if image_file_path is not None:
            os.remove(image_file_path)
        return image_name
    return None


label_mapping = {1: 0, 3: 1, 6: 2, 7: 3}


input_folder = 'MMPP_data/Validation/Annotations/TOA/5.Mainroad_F01/'
output_folder = "MMPP_data/5.Mainroad_F01_del"


image_folder = "MMPP_data/Validation/Images/TOA/5.Mainroad_F01/"


if not os.path.exists(output_folder):
    os.makedirs(output_folder)

json_files = [f for f in os.listdir(input_folder) if f.endswith('.json')]


deleted_files = []

for json_file in json_files:
    json_file_path = os.path.join(input_folder, json_file)
    
    deleted_file = json_to_yolo(json_file_path, output_folder, label_mapping, image_folder)

    
    if deleted_file:
        deleted_files.append(deleted_file)

print("삭제된 파일 목록:")
for deleted_file in deleted_files:
    print(deleted_file)